In [82]:
import pandas as pd
import numpy as np
import plotly.express as px

In [134]:
def cargar_portfolio_admin(path):
    portfolio = pd.read_csv(path) # Carga .csv de admin
    portfolio = portfolio[['Folio','Principal a pagar','Fecha de Creación','Colateral restante','Colateral inicial','Original liquid principal amount cents','Accrued interest amount cents','Paid interest amount cents']].set_index('Folio')
    portfolio['Principal a pagar'] = portfolio['Principal a pagar'] / 100
    portfolio['Original liquid principal amount cents'] = portfolio['Original liquid principal amount cents'] / 100
    portfolio['Accrued interest amount cents'] = portfolio['Accrued interest amount cents'] / 100
    portfolio['Paid interest amount cents'] = portfolio['Paid interest amount cents'] / 100
    portfolio['Colateral restante'] = portfolio['Colateral restante'] / 10**8
    portfolio['Colateral inicial'] = portfolio['Colateral inicial'] / 10**8

    portfolio['Precio inicial credito'] = portfolio['Original liquid principal amount cents'] / (portfolio['Colateral inicial'] * .5)
    portfolio['Precio MarginCall'] = portfolio['Principal a pagar'] / (portfolio['Colateral restante'] * .7)
    portfolio['Precio liquidacion'] = portfolio['Principal a pagar'] / (portfolio['Colateral restante'] * .8)
    return portfolio

In [211]:
# caragr portfolio de créditos con csv de admin
portfolio = cargar_portfolio_admin('/Users/cristobalcordova/Downloads/loans-2021-11-30.csv')
portfolio_1 = cargar_portfolio_admin('/Users/cristobalcordova/Downloads/loans-2021-12-09.csv')

In [151]:
# principal a pagar: lo que debe (capita + intereses)
# Original liquid principal amount cents: crédito original

In [212]:
# Total prestado antes t = 0
portfolio['Principal a pagar'].sum()

6802112.4

In [213]:
# total prestado despues t = 1
portfolio_1['Principal a pagar'].sum()

4812210.62

In [219]:
# dieferencia de lo que tengo prestado o liquidado en clp
liquidado_clp = portfolio['Principal a pagar'].sum() - portfolio_1['Principal a pagar'].sum()
liquidado_clp

1989901.7800000003

In [218]:
# precio ponderado de liquidación teorico discreto
# (portfolio["Precio liquidacion"] * portfolio["Colateral restante"]).sum() / portfolio["Colateral restante"].sum()

In [194]:
# diferencia colateral prestado antes - colateral prestado despues, colateral liquidado btc
liq_colateral = portfolio['Colateral restante'].sum() - portfolio_1['Colateral restante'].sum()
liq_colateral

0.058946739999999886

In [215]:
portfolio_1['Colateral restante'].sum()

0.21370078000000003

In [216]:
# precio promedio de la liquidación
(portfolio['Principal a pagar'].sum() - portfolio_1['Principal a pagar'].sum()) / (portfolio['Colateral restante'].sum() - portfolio_1['Colateral restante'].sum())

34185454.07567418

precio_prom_liq =  liquidado / liq_colateral

In [189]:
precio_max = portfolio_1['Precio inicial credito'].max()
precio_min = portfolio_1['Precio inicial credito'].min()
precio_pro = portfolio_1['Precio inicial credito'].mean()

print('Precio máximo: %s' %(precio_max))
print('Precio minimo: %s' %(precio_min))
print('Precio promedio: %s' %(precio_pro))

Precio máximo: 66398725.67565683
Precio minimo: 22892717.78483121
Precio promedio: 46592391.788525


In [156]:
class cartera():

    def __init__(self) -> None:
        
        self.creditos = pd.DataFrame(columns=['Principal a pagar', 'Colateral restante'])

    def add_credito(self, _prestamo, _colateral):

        self.creditos = self.creditos.append(pd.DataFrame([[_prestamo, _colateral]], columns=['Prestamo', 'Colateral']), ignore_index=True)

    def add_creditos(self, df):

        self.creditos = self.creditos.append(df)

    def ltv(self, price):
        self.creditos['LTV'] = self.creditos['Principal a pagar'] / (self.creditos['Colateral restante'] * price)

    def simulacion_crash(self, p, price):
        precio_crash = price * (1 - p) # price debe ser input
        df_sim = self.creditos
        # calculo nuevo ltv con caide del p%
        df_sim['LTV_fin'] = df_sim['Principal a pagar']/(df_sim['Colateral restante'] * price * (1 - p))
        df_sim = df_sim[df_sim['LTV_fin']>= .8]

        # principal_tarjet: 
        # amount: lo que tengo q liquidar
        df_sim['principal_target'] = (0.65 * df_sim['Colateral restante'] * precio_crash)
        df_sim['amount'] = (df_sim['Principal a pagar'] - df_sim['principal_target']) / (1-0.65)
        # df_sim['amount'] = df_sim[['Prestamo', 'amount']].apply(lambda x: x['Prestamo'] if x['Prestamo']<x['amount'] else x['amount'], axis=1)

        # como quedan los creditos despues de la liquidación
        # df_sim['principal_target'].sum()
        df_sim['Prestamo_after_liq'] = (df_sim['Principal a pagar'] - df_sim['amount'])
        df_sim['LTV_after_liq'] = (df_sim['Principal a pagar'] - df_sim['amount']) / (df_sim['Colateral restante'] * precio_crash -df_sim['amount'])
        self.cart_crash = df_sim
        self.cartera_to_liquidate = round(df_sim['amount'].sum())

In [220]:
df = portfolio[['Principal a pagar', 'Colateral restante']]
df1 = portfolio_1[['Principal a pagar', 'Colateral restante']]


In [167]:
(df['Colateral restante']-df1['Colateral restante']).sum()

0.058946740000000004

In [221]:
# creo cartera con créditos
cart = cartera()
cart.add_creditos(df)
cart.ltv(48000000)

cart1 = cartera()
cart1.add_creditos(df1)
cart1.ltv(48000000)
prestado = cart.creditos['Principal a pagar'].sum()

In [222]:
simulaciones = pd.DataFrame(columns= ['Promedio prestado', 'Colateral liquidado', 'Caida'])
precio_sim = 48000000
for p in range(20, 41, 1):
    cart.simulacion_crash(p/100, precio_sim)
    liquidado = cart.cart_crash['amount'].sum()
    simulaciones = simulaciones.append(pd.DataFrame(np.array([[prestado,liquidado,p]]),columns= ['Promedio prestado', 'Colateral liquidado', 'Caida']))

/var/folders/y5/6qkmm8_91817rs0kg5vpd5vm0000gn/T/ipykernel_53289/3486115452.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/y5/6qkmm8_91817rs0kg5vpd5vm0000gn/T/ipykernel_53289/3486115452.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/y5/6qkmm8_91817rs0kg5vpd5vm0000gn/T/ipykernel_53289/3486115452.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [223]:
# simulaciones
simulaciones['Relacion liquidado/prestado'] = simulaciones['Colateral liquidado'] / simulaciones['Promedio prestado']
px.line(simulaciones, x='Caida', y='Colateral liquidado', title='Monto liquidado, ante caida desde los ' +(str(precio_sim)))

In [224]:
# liquidacion segun modelo caida discreta
#
liquiado_model = simulaciones[simulaciones['Caida']== 35]['Colateral liquidado'].loc[0]
# Error
(liquiado_model - liquidado_clp) / liquiado_model

0.2593823179044683

In [225]:
liquiado_model

2686813.761142857